# Code Profiling

Code profiling is the process of dynamically analyzing the performance of your code or software, focusing on aspects like runtime, memory allocation, and function execution frequency. Its primary goal is to identify performance bottlenecks, excessive resource consumption, and areas that take too long to execute, enabling developers to optimize their code for better efficiency and speed. 

Here we use a simple Python profiler called "cProfile", which is built into the Python standard library. It can be used to assess the runtime and execution frequency of all functions utilized in a script. Let's look at an example below!

# Running cProfile

First, create a Python script that repeatedly calls the function or class you want to speed up a few hundred times atleast (this helps average out any variations in execution time due to different input parameters or simply makes sure the most time consuming functions are reflected as such in the output file). Then, run the following command from the command line:

`python -m cProfile -o profiler_output.txt profile_orbitize.py`

- The `-m cProfile` will attach the cProfile profiler to this script when it runs.

- The `-o profiler_output.txt` will save its output to "profiler_output.txt" (otherwise it dumps the output stats into stdout).

_Note on parallelization: Using parallelism will make the cprofiler output more confusing than necessary. We just want to know what takes the most time,
summed up across multiple processes. Doing this on parallelized code, each process must save its own cProfile output and you 
need to combine the outputs afterwards (using `pstats.add()`)._

# Analyzing the cProfile output

After you have run the `profile_orbitize.py` function with the cProfile package, we will use the `pstats` module (part of Python standard library) to read and analyze the output of the profiler (a binary file) and identify bottlenecks in the code. If you are in Colaboratory, you will need to upload the output of the cProfile to Colaboratory.


In [1]:
import pstats

mystats = pstats.Stats("profiler_output.txt")

## Display the top 15 functions that took the longest time to run

Even in such a short script, python called millions of function! A lot of them are super quick and really not what we are after. Typically, we want to find the few functions that are bottlenecks in our code.

The function below sorts the functions by which took the longest to run, and prints out only the stats of the top 15 functions that took longest to run. The columns below mean:
  * ncalls: number of times this function was called
  * tottime: the total execution time spent in this code NOT including calls to other functions
  * percall: this first percall divides tottime by ncalls. The amount of time per call spent solely in this function.
  * cumtime: the total execution time spent in this code INCLUDING calls to sub functions. Note: <u>cumtime > tottime always</u>.
  * percall: second percall divides cumtime by ncalls
  * filename: the name of the function being considered

Both `tottime` and `cumtime` are useful. A function with a high `tottime` means we should focus on speeding up this function. A function with only a high `cumtime` means we should see what this function is calling to improve runtime.
  * `_newton_solver()` is a function with a high tottime. We should look at the lines in that function to check out how to speed it up.
  * `compute_model()` is a function with a low tottime but a very high `cumtime`. We should look at the functions it calls to figure out what takes up so much time.

Generally, we want to put some filters on `print_stats`, becuase otherwise there will be so much printed out it is unmanageable.

For more features: [pstats documentaiton](https://docs.python.org/3/library/profile.html)

In [8]:
mystats.sort_stats("time").print_stats(30)

Fri Jul 12 19:25:45 2024    profiler_output.txt

         17846372 function calls (17690023 primitive calls) in 116.584 seconds

   Ordered by: internal time
   List reduced from 9826 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4988   30.573    0.006   30.707    0.006 /Users/kruthik/orbitize/orbitize/kepler.py:199(_newton_solver_wrapper)
     4988   26.036    0.005   74.896    0.015 /Users/kruthik/orbitize/orbitize/kepler.py:48(calc_orbit)
     4988    7.440    0.001    7.440    0.001 /Users/kruthik/orbitize/orbitize/system.py:721(radec2seppa)
     4988    7.288    0.001   39.182    0.008 /Users/kruthik/orbitize/orbitize/kepler.py:143(_calc_ecc_anom)
     2494    4.488    0.002    4.547    0.002 /Users/kruthik/orbitize/orbitize/lnlike.py:8(chi2_lnlike)
     4988    3.609    0.001    6.345    0.001 /Users/kruthik/orbitize/orbitize/kepler.py:18(tau_to_manom)
     2494    3.324    0.001   34.476    0.014 /Users/kruthik/orbiti

We can also apply multiple filters. For example let's look at the top 15 numpy functions that took the most time to run. Note that the order of the filtering matters. This command first selects every function with numpy in the name, followed by taking the top 15. Calling `print_stats(15, 'numpy')` would pick the top 15 longest runtime functions, and downselecting which has numpy in the name from those. That would give us less than 15 numpy functions.

In [3]:
mystats.sort_stats("time").print_stats("numpy", 15)

Fri Jul 12 19:25:45 2024    profiler_output.txt

         17846372 function calls (17690023 primitive calls) in 116.584 seconds

   Ordered by: internal time
   List reduced from 9826 to 680 due to restriction <'numpy'>
   List reduced from 680 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    22474    1.360    0.000    1.360    0.000 {built-in method numpy.zeros}
   255614    0.491    0.000    0.491    0.000 {built-in method numpy.array}
    35832    0.439    0.000    0.439    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    14964    0.170    0.000    0.170    0.000 {built-in method numpy._core._multiarray_umath._place}
    17555    0.151    0.000    0.151    0.000 {built-in method numpy.asarray}
    29928    0.149    0.000    0.193    0.000 /Users/kruthik/opt/miniconda3/envs/codeastro2024/lib/python3.10/site-packages/numpy/lib/_stride_tricks_impl.py:349(_broadcast_to)
     2494    0.148    0.000    0.223    0.000 /Users/kru

In [9]:
mystats.sort_stats("time").print_stats(15, "numpy")

Fri Jul 12 19:25:45 2024    profiler_output.txt

         17846372 function calls (17690023 primitive calls) in 116.584 seconds

   Ordered by: internal time
   List reduced from 9826 to 15 due to restriction <15>
   List reduced from 15 to 1 due to restriction <'numpy'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    22474    1.360    0.000    1.360    0.000 {built-in method numpy.zeros}




# Activities

## Activity Part 1: Analyze the output

Generate the output above on your own by running `python -m cProfile -o profiler_output.txt profile_orbitize.py` and answer the following questions by analyzing the output.

1. Which function takes up the most runtime not including calls to sub-functions?

2. Which function takes up the most runtime including calls to sub-functions?

3. Which function is called the most? Which `orbitize` function is called the most?

4. If we had the magical ability to speed up one of the functions in the previous three answers by a factor of 2, which function should we speed up? What is the improvement in end-to-end runtime of the script?


_Note: Depending on your system, some inbuilt functions might turn out to be the most time consuming ones. But here we are looking for the ones we can speed up, namely the orbitize functions._


## Activity Part 2: Investigate why `_logl` takes so long

`_logl()` is a helper function in `orbitize!` to compute the log likelihood of the data given the model. The `_logl()` function itself has a short runtime but it is calling something that takes a long time that makes it have a long `cumtime`. We can use the `print_callees()` function to look at the stats of all the functions it calls.

_Hint: We can see that `compute_model` is the function with the highest cumtime, but its tottime is low, so we know something it calls takes all the time. We must dig deeper! Keep digging down recursively to find what function called within `_logl()` that takes the longest._

In [4]:
mystats.print_callees("_logl")

   Ordered by: internal time
   List reduced from 9826 to 1 due to restriction <'_logl'>

Function                                               called...
                                                           ncalls  tottime  cumtime
/Users/kruthik/orbitize/orbitize/sampler.py:53(_logl)  ->    9976    0.006    0.014  /Users/kruthik/opt/miniconda3/envs/codeastro2024/lib/python3.10/site-packages/astropy/table/table.py:2091(__getitem__)
                                                             2494    0.001    0.001  /Users/kruthik/opt/miniconda3/envs/codeastro2024/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:635(_nansum_dispatcher)
                                                             2494    0.007    0.409  /Users/kruthik/opt/miniconda3/envs/codeastro2024/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:640(nansum)
                                                             2494    4.488    4.547  /Users/kruthik/orbitize/orbitize/lnlike.py:8(c

## Activity Part 3: Which function calls `numpy.array` the most?

`numpy.array` is a popular function because it gets called anytime a new numpy array gets created. We can use `print_callers()` to see which functions call it to look into potentially reducing the number of array creations to speed up the code. Which function in `orbitize` calls `numpy.array` the most times per function call (not including calls from subfunctions)?

In [6]:
mystats.print_callers("numpy.array")

   Ordered by: internal time
   List reduced from 9826 to 1 due to restriction <'numpy.array'>

Function                       was called by...
                                   ncalls  tottime  cumtime
{built-in method numpy.array}  <-      77    0.000    0.000  /Users/kruthik/opt/miniconda3/envs/codeastro2024/lib/python3.10/site-packages/astropy/constants/constant.py:123(__new__)
                                        2    0.000    0.000  /Users/kruthik/opt/miniconda3/envs/codeastro2024/lib/python3.10/site-packages/astropy/coordinates/builtin_frames/fk4_fk5_transforms.py:1(<module>)
                                        2    0.000    0.000  /Users/kruthik/opt/miniconda3/envs/codeastro2024/lib/python3.10/site-packages/astropy/coordinates/transformations/affine.py:295(__init__)
                                        6    0.000    0.000  /Users/kruthik/opt/miniconda3/envs/codeastro2024/lib/python3.10/site-packages/astropy/io/ascii/core.py:975(convert_numpy)
                        

# Some Ways to Speed up Your Code

If you identified the critical chunk of code to speed up, the general strategy is to remove unncessary computations as much as possible. There's unfortunately not one single method to speed up your code. But here are a few ideas:

1. Eliminate computations that are not being used (e.g., computing variables you do not use; processing parts of images that will be discarded). 
2. Use `numpy` functions whenever possible for matrix operations
3. Avoid using python `for` and `while` loops (see example below)
4. If MKL/BLAS is being run inside of already-parallelized code, disable MKL/BLAS (see Day 1 parallelization materials for more details on MKL/BLAS).
5. Reduce the number of iterations or increase the tolerance of routines that are unncessarily precise (e.g., optimizers can run for less iterations; sin and cos can be approximated by taylor expansions). 
6. Avoid copying variables when they do not need to be copied (e.g., if the input is already a numpy.array, you don't need to wrap it with `np.array()` to ensure it's a numpy array). 
7. Turn some of your python code into C-code and call it with Python

Any other ideas?


In [10]:
"""
Here's an example of replacing a for loop with a faster masked operation.
This is called VECTORIZATION.
"""

import numpy as np

# add 5 to each value in a list that's greater than 6
arr = np.asarray([1, 4, 6, 7, 8, 10, 2])
arr_2 = np.asarray([1, 4, 6, 7, 8, 10, 2])


# first option (much slower for long arrays)
for i, elem in enumerate(arr):
    if elem > 6:
        arr[i] += 5

# second option (faster)
mask_arr = arr_2 > 6
arr_2[mask_arr] += 5

# print resulting arrays for both cases
print(arr)
print(arr_2)

[ 1  4  6 12 13 15  2]
[ 1  4  6 12 13 15  2]


# Code Optimizing Strategy

![](imgs/code_speedup.png)

- Generally, it is important to focus your effort on speeding up the 1 or 2 slowest functions. If you speed up something that takes 50% of the runtime by a factor of 2, you speed up the code by a factor of 25%. If a piece of code takes 10% of the runtime, a factor of 2 improvement (which by itself is often hard to get) only gives you a 5% improvement in the end. **Focus your effort on optimizing the runtime of the one or two functions that take the most amount of time.**

- For all else, it is generally more important to optimize code readibility over runtime. Code that is more readible is less prone to bugs and eaiser to maintain. If a piece of code takes only 0.0001% of the runtime, any amount of speed up is not worth making the code hard to read (from a person-hour perspective). Many times, it is difficult to gain more than a factor of 2 speed up in runtime. __Definitely think about the potential gain in speed-up versus how long it takes to write and maintian the code__.



## Extra: Code Performance Extrapolation

If your code takes 24 hours to run to completion, or needs to be run on a cluster, it is likely still easier to develop and benchmark it on your laptop. We can use extrapolation to estimate performance. 

### Runtime

Generally algorithm runtimes scale as N^a where a is some positive number greater than 1, and N is the size of your data. (In CS terms, this is often disccused in "Big O" notation as  $O(N^a)$ runtime). If your data is too big or takes too long to benchmark on a single machine, try running your code on much smaller data. If you do this for ~3-5 different test data with different N, you can estimate what is the scaling of your code (i.e., what is "a"). Once you know the scaling you can use that scaling relation to extrapolate how long it will take on larger datasets. If you can also use more cores, you can divide the final runtime by the number of cores you can use. 

For example (very idealized), my code takes 1 second on 10x10 data, 100 seconds on 100x100 data, and 400 seconds on 200x200 data (all on a single core). My code scales as (N/10)^2 where N is the size of one dimension of the data. If I want to run it on 10000x10000 data with 100 cores, my code should take (1000^2)/100 = 10,000 seconds to run. In practice there will probably be some deviations to these estimates (due to overheads, CPU usage scheudling, other factors), but this scaling should give you a general sense of how long something will take. 

### Memory

If you are worried that your program will run out of memory, try calculating how much memory your program uses. On modern 64-bit machines, a single number (an integer, float) takes up 8 bytes. If you have an array of 1000x1000 floats, then it will take up 8e6 bytes or 8 MBs. If you have 3-D numerical data with dimensions 1000x1000x1000, then it will take up 8e9 bytes or 8 GBs.